In [1]:
#Uploading required packages
import numpy as np
from scipy.stats import gamma
from scipy.stats import uniform as unif
from scipy.stats import multivariate_normal as mvtnorm
from scipy.stats import bernoulli
from numba import jit
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
# Indian Buffet Process Function

def sampleIBP(alpha, num_objects):  
    # Initializing storage for results
    result = np.zeros([num_objects, 1000])
    # Draw from the prior for alpha
    t = np.random.poisson(alpha)
    # Filling in first row of result matrix
    result[0, 0:t] = np.ones(t) #changed form np.ones([1, t])
    # Initializing K+
    K_plus = t
    
    for i in range(1, num_objects):
        for j in range(0, K_plus):
            p = np.array([np.log(np.sum(result[0:i,j])) - np.log(i+1), 
                          np.log(i+1 - np.sum(result[0:i, j])) - np.log(i+1)])
            p = np.exp(p - max(p))

            if(np.random.uniform() < p[0]/np.sum(p)):
                result[i, j] = 1
            else:
                result[i, j] = 0
        t = np.random.poisson(alpha/(i+1))
        x = K_plus + 1
        y = K_plus + t
        result[i, (x-1):y] = np.ones(t) #changed form np.ones([1, t])
        K_plus = K_plus+t
    result = result[:, 0:K_plus]
    
    return list([result, K_plus])

In [18]:
#Data Simulation

#Latent Features
W = np.array([[0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]])


#Each image in our simulated data set is the superposition of four base images#
# Number of images/ data points
num_objects=100

#Dimension of image (6x6)
object_dim  = 6*6

#Covariance matrix for images/ white noise
sigma_x_orig = 0.5
I = sigma_x_orig * np.identity(object_dim)

#z_i - binary feature matrix (1 x 4) - each entry set to 1 with probability 0.5 and 0 otherwise#
#x is data variable - each row correspondes to a superimposed built from a random combination of latent features#
#with white noise added - x is built with multivariate gaussian#
image_data = np.zeros((100,36))
z_org = np.zeros((100,4))

for i in range(0,num_objects):
    z_org[i,:] = np.array([bernoulli.rvs(p=0.5, size=4)])
    image_data[i,:] = np.dot(z_org[i,:],W) + np.random.normal(0,1, (1,object_dim)).dot(I) 

In [22]:
## Optimized likelihood
# This function return the log likelihood
def likelihood_opt(X, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim):
    #Calculate M
    M = np.dot(Z.T, Z) + (sigma_X**2/sigma_A**2)*np.eye(K_plus)
    
    part1 = (-1)*num_objects*(0.5*object_dim)*np.log(2*np.pi)
    part2 = (-1)*(num_objects-K_plus)* object_dim *np.log(sigma_X) 
    part3 = (-1)*object_dim*K_plus*np.log(sigma_A) 
    part4 = (-1)*(0.5*object_dim)* np.log(np.linalg.det(M)) 
    part5 = (-1/(2*sigma_X**2)) * np.trace(np.dot(np.dot(X.T,(np.identity(num_objects) - np.dot(np.dot(Z,np.linalg.inv(M)),Z.T))),X))
    total = part1+part2+part3+part4+part5
    return(total)

Naive
====

In [23]:
def sampler_naive(image_data, num_objects, object_dim, E=1000):

    data = image_data

    #Set truncation limit for max number of sampled latent features
    K_inf = 20

    #Set storage arrays for sampled parameters
    chain_Z = np.zeros([E, num_objects, K_inf])
    chain_K = np.zeros([E, 1])
    chain_sigma_X = np.zeros([E, 1])
    chain_sigma_A = np.zeros([E, 1])
    chain_alpha = np.zeros([E, 1])

    #Initialize parameter values
    sigma_X = 1
    sigma_A = 1
    alpha = 1
    num_object= np.shape(data)[0]
    object_dim = np.shape(data)[1]
    [Z, K_plus] = sampleIBP(alpha, num_objects)

    #Compute Harmonic Number
    HN = 0
    for i in range(0, num_objects):
        HN = HN + 1/(i+1)

    for e in range(0, E):

        #Store sampled values
        chain_Z[e, :, 0:K_plus] = Z[:, 0:K_plus]
        chain_K[e] = K_plus
        chain_sigma_X[e] = sigma_X
        chain_sigma_A[e] = sigma_A
        chain_alpha[e] = alpha

        if (e%100==0):
            print(e)

        print("At iteration", e, ": K_plus is", K_plus, ", alpha is", alpha) 

        #Generate a new value for Z[i,k] and accept by Metropolis
        for i in range(0, num_objects):
            #First we remove singular features if any
            for k in range(0, K_plus):
                if (k>=K_plus):
                    break
                if(Z[i, k] > 0):
                    if (np.sum(Z[:, k]) - Z[i, k]) <= 0: 
                        Z[i, k] = 0
                        Z[:, k:(K_plus - 1)] = Z[:, (k+1):K_plus]
                        K_plus = K_plus - 1
                        Z = Z[:, 0:K_plus]
                        continue
                #Compute conditional distribution for current cell
                P=np.zeros(2)

                Z[i,k]=1
                P[0] = likelihood_opt(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(np.sum(Z[:, k]) - Z[i,k]) - np.log(num_objects)

                Z[i,k]=0
                P[1] = likelihood_opt(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(num_objects - np.sum(Z[:,k])) - np.log(num_objects)

                P = np.exp(P - np.max(P))

                #Sample from conditional(sampling previously sampled dishes)
                if np.random.uniform(0,1) < (P[0]/(np.sum(P))):
                    Z[i, k] = 1
                else:
                    Z[i, k] = 0

            #Sample new dishes by Metropolis
            trunc = np.zeros(5)
            alpha_N = alpha/num_objects

            for k_i in range(0,5):
                Z_temp = Z
                if k_i>0:
                    newcol = np.zeros((num_objects, k_i))
                    newcol[i,:] = 1 
                    Z_temp = np.column_stack((Z_temp, newcol))
                trunc[k_i] = k_i * np.log(alpha_N) - alpha_N - np.log(np.math.factorial(k_i)) + likelihood_opt(data, Z_temp,sigma_A, sigma_X, K_plus+k_i, num_objects, object_dim)

            trunc = np.exp(trunc - np.max(trunc))
            trunc = trunc/np.sum(trunc)

            p = np.random.uniform(0,1)
            t = 0
            new_dishes = 0

            for k_i in range(0,5):
                t = t + trunc[k_i]
                if p < t:
                    new_dishes = k_i
                    break

            if(new_dishes > 0):
                newcol = np.zeros((num_objects, new_dishes))
                newcol[i,:] = 1
                Z = np.column_stack((Z, newcol))
            K_plus = K_plus + new_dishes

        #Sample sigma_X and sigma_A through Metropolis
        lik_curr = likelihood_opt(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim)

        if np.random.uniform(0,1) < 0.5:
            sigma_X_new = sigma_X - np.random.uniform(0,1)/20
        else:
            sigma_X_new = sigma_X + np.random.uniform(0,1)/20

        lik_new_X = likelihood_opt(data, Z, sigma_A, sigma_X_new, K_plus, num_objects, object_dim)

        acc_X = np.exp(min(0, lik_new_X - lik_curr))

        if np.random.uniform(0,1) < 0.5:
            sigma_A_new = sigma_A - np.random.uniform(0,1)/20
        else:
            sigma_A_new = sigma_A + np.random.uniform(0,1)/20

        lik_new_A = likelihood_opt(data, Z, sigma_A_new, sigma_X, K_plus, num_objects, object_dim)

        acc_A = np.exp(min(0, lik_new_A - lik_curr))

        if np.random.uniform(0,1) < acc_X:
            sigma_X = sigma_X_new
        if np.random.uniform(0,1) < acc_A:
            sigma_A = sigma_A_new

        #Sample alpha via Gibbs
        alpha = np.random.gamma(1 + K_plus, 1/(1+HN))

    print("Complete")
    return(chain_K)

In [26]:
%%time
np.random.seed(1234)
V = sampler_naive(image_data, num_objects, object_dim, E=200)

0
At iteration 0 : K_plus is 10 , alpha is 1
At iteration 1 : K_plus is 4 , alpha is 0.29502510496294904
At iteration 2 : K_plus is 2 , alpha is 0.1780099709034097
At iteration 3 : K_plus is 2 , alpha is 0.2256886341975698
At iteration 4 : K_plus is 2 , alpha is 0.7546239841720612
At iteration 5 : K_plus is 2 , alpha is 0.6432954795108905
At iteration 6 : K_plus is 2 , alpha is 0.3881116193355114
At iteration 7 : K_plus is 2 , alpha is 0.08115084323003742
At iteration 8 : K_plus is 2 , alpha is 0.29438778165828117
At iteration 9 : K_plus is 2 , alpha is 0.33512708312924583
At iteration 10 : K_plus is 2 , alpha is 0.3116294218683443
At iteration 11 : K_plus is 2 , alpha is 0.16718152872178224
At iteration 12 : K_plus is 2 , alpha is 0.5207559578618401
At iteration 13 : K_plus is 2 , alpha is 0.3367866096645625
At iteration 14 : K_plus is 2 , alpha is 0.9111640776047102
At iteration 15 : K_plus is 2 , alpha is 0.35103637419458256
At iteration 16 : K_plus is 2 , alpha is 0.327999950790827

Cython
====

In [27]:
%load_ext cython

In [28]:
%%cython -a

import numpy as np
cimport numpy as np
import cython
from libc.math cimport log

cdef double pi = 3.141592653589793

cdef sampleIBP_c(double alpha, long num_objects):  
    cdef long i, j
    cdef long t
    cdef long x
    cdef long y
    cdef long K_plus

    # Initializing storage for results
    result = np.zeros([num_objects, 1000])

    # Draw from the prior for alpha
    t = np.random.poisson(alpha)
    # Filling in first row of result matrix
    result[0, 0:t] = np.ones(t) #changed form np.ones([1, t])
    # Initializing K+
    K_plus = t
    
    for i in range(1, num_objects):
        for j in range(0, K_plus):
            p = np.array([log(np.sum(result[0:i,j])) - log(i+1), 
                          log(i+1 - np.sum(result[0:i, j])) - log(i+1)])
            p = np.exp(p - max(p))

            if(np.random.uniform() < p[0]/np.sum(p)):
                result[i, j] = 1
            else:
                result[i, j] = 0
        t = np.random.poisson(alpha/(i+1))
        x = K_plus + 1
        y = K_plus + t
        result[i, (x-1):y] = np.ones(t) #changed form np.ones([1, t])
        K_plus = K_plus+t
    result = result[:, 0:K_plus]

    return list([result, K_plus])

cdef double likelihood_opt_c(double[:,:] X, double[:, :] Z, double sigma_A, double sigma_X, 
                     long K_plus, long num_objects, long object_dim):

    #Calculate M
    cdef double[:, :] M = np.dot(Z.T, Z) + (sigma_X**2/sigma_A**2)*np.eye(K_plus)
    cdef double part1, part2, part3, part4, part5, total

    part1 = (-1)*num_objects*(0.5*object_dim)*log(2*pi)
    part2 = (-1)*(num_objects-K_plus)* object_dim *log(sigma_X) 
    part3 = (-1)*object_dim*K_plus*log(sigma_A) 
    part4 = (-1)*(0.5*object_dim)* log(np.linalg.det(M)) 
    part5 = (-1/(2*sigma_X**2)) * np.trace(np.dot(np.dot(X.T,(np.identity(num_objects) - np.dot(np.dot(Z,np.linalg.inv(M)),Z.T))),X))
    total = part1+part2+part3+part4+part5

    return(total)

def sampler_naive_c(double[:, :] image_data, long num_objects, long object_dim, E=1000):

    cdef double[:, :] data = image_data

    #Set truncation limit for max number of sampled latent features
    cdef long K_inf = 20

    #Set storage arrays for sampled parameters
    cdef double[:, :, :] chain_Z = np.zeros([E, num_objects, K_inf])
    cdef double[:, :] chain_K = np.zeros([E, 1])
    cdef double[:, :] chain_sigma_X = np.zeros([E, 1])
    cdef double[:, :] chain_sigma_A = np.zeros([E, 1])
    cdef double[:, :] chain_alpha = np.zeros([E, 1])

    #Initialize parameter values
    cdef double sigma_X = 1
    cdef double sigma_A = 1
    cdef double alpha = 1
    cdef long num_object= np.shape(data)[0]
    cdef long cbject_dim = np.shape(data)[1]

    [Z, K_plus] = sampleIBP_c(alpha, num_objects)

    #Compute Harmonic Number
    cdef double HN = 0

    cdef long i, j1, j2, e, k, k_i

    for i in range(0, num_objects):
        HN = HN + 1.0/(i+1)

    for e in range(0, E):
        #Store sampled values
        for j1 in range(num_objects):
            for j2 in range(K_plus):
                chain_Z[e, j1, j2] = Z[j1, j2]
        # chain_Z[e, :, 0:K_plus] = Z[:, 0:K_plus]
        chain_K[e] = K_plus
        chain_sigma_X[e] = sigma_X
        chain_sigma_A[e] = sigma_A
        chain_alpha[e] = alpha

        if (e%100==0):
            print(e)

        print("At iteration", e, ": K_plus is", K_plus, ", alpha is", alpha) 

        #Generate a new value for Z[i,k] and accept by Metropolis
        for i in range(0, num_objects):
            #First we remove singular features if any

            for k in range(0, K_plus):

                if (k>=K_plus):
                    break
                if(Z[i, k] > 0):
                    if (np.sum(Z[:, k]) - Z[i, k]) <= 0: 
                        Z[i, k] = 0
                        for j1 in range(num_objects):
                            for j2 in range(k, K_plus - 1):
                                Z[j1, j2] = Z[j1, j2+1]
                        # Z[:, k:(K_plus - 1)] = Z[:, (k+1):K_plus]
                        K_plus = K_plus - 1
                        Z = Z[:, 0:K_plus]
                        continue
                #Compute conditional distribution for current cell
                P=np.zeros(2)
                
                Z[i,k]=1
                P[0] = likelihood_opt_c(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + log(np.sum(Z[:, k]) - Z[i,k]) - log(num_objects)

                Z[i,k]=0
                P[1] = likelihood_opt_c(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + log(num_objects - np.sum(Z[:,k])) - log(num_objects)

                P = np.exp(P - np.max(P))

                #Sample from conditional(sampling previously sampled dishes)
                if np.random.uniform(0,1) < (P[0]/(np.sum(P))):
                    Z[i, k] = 1
                else:
                    Z[i, k] = 0

            #Sample new dishes by Metropolis
            trunc = np.zeros(5)
            alpha_N = alpha/num_objects

            for k_i in range(0,5):
                Z_temp = Z
                if k_i>0:
                    newcol = np.zeros((num_objects, k_i))
                    newcol[i,:] = 1 
                    Z_temp = np.column_stack((Z_temp, newcol))
                trunc[k_i] = k_i * log(alpha_N) - alpha_N - log(np.math.factorial(k_i)) + likelihood_opt_c(data, Z_temp,sigma_A, sigma_X, K_plus+k_i, num_objects, object_dim)

            trunc = np.exp(trunc - np.max(trunc))
            trunc = trunc/np.sum(trunc)

            p = np.random.uniform(0,1)
            t = 0
            new_dishes = 0

            for k_i in range(0,5):
                t = t + trunc[k_i]
                if p < t:
                    new_dishes = k_i
                    break

            if(new_dishes > 0):
                newcol = np.zeros((num_objects, new_dishes))
                newcol[i,:] = 1
                Z = np.column_stack((Z, newcol))
            K_plus = K_plus + new_dishes

        #Sample sigma_X and sigma_A through Metropolis
        lik_curr = likelihood_opt_c(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim)

        if np.random.uniform(0,1) < 0.5:
            sigma_X_new = sigma_X - np.random.uniform(0,1)/20
        else:
            sigma_X_new = sigma_X + np.random.uniform(0,1)/20

        lik_new_X = likelihood_opt_c(data, Z, sigma_A, sigma_X_new, K_plus, num_objects, object_dim)

        acc_X = np.exp(min(0, lik_new_X - lik_curr))

        if np.random.uniform(0,1) < 0.5:
            sigma_A_new = sigma_A - np.random.uniform(0,1)/20
        else:
            sigma_A_new = sigma_A + np.random.uniform(0,1)/20

        lik_new_A = likelihood_opt_c(data, Z, sigma_A_new, sigma_X, K_plus, num_objects, object_dim)

        acc_A = np.exp(min(0, lik_new_A - lik_curr))

        if np.random.uniform(0,1) < acc_X:
            sigma_X = sigma_X_new
        if np.random.uniform(0,1) < acc_A:
            sigma_A = sigma_A_new

        #Sample alpha via Gibbs
        alpha = np.random.gamma(1 + K_plus, 1/(1+HN))

    print("Complete")
    return(chain_K)

DistutilsPlatformError: Unable to find vcvarsall.bat

In [ ]:
%%time
np.random.seed(123)
x = sampler_naive(image_data, num_objects, object_dim, E=20)

In [ ]:
%%time
np.random.seed(123)
x = sampler_naive_c(image_data, num_objects, object_dim, E=20)

In [ ]:
%%cython -a

import numpy as np
from scipy.stats import gamma
from scipy.stats import uniform as unif
from scipy.stats import multivariate_normal as mvtnorm
from scipy.stats import bernoulli
cimport cython

def sampleIBP1(double alpha, int num_objects):  
    
    #cdef int i, j, k
    #cdef int K_plus
    # Initializing storage for results
    result = np.zeros([num_objects, 1000])
    # Draw from the prior for alpha
    t = np.random.poisson(alpha)
    # Filling in first row of result matrix
    result[0, 0:t] = np.ones(t) #changed form np.ones([1, t])
    # Initializing K+
    K_plus = t
    
    for i in range(1, num_objects):
        for j in range(0, K_plus):
            p = np.array([log(np.sum(result[0:i,j])) - log(i+1), 
                          log(i+1 - np.sum(result[0:i, j])) - log(i+1)])
            p = np.exp(p - max(p))

            if(np.random.uniform() < p[0]/np.sum(p)):
                result[i, j] = 1
            else:
                result[i, j] = 0
        t = np.random.poisson(alpha/(i+1))
        x = K_plus + 1
        y = K_plus + t
        result[i, (x-1):y] = np.ones(t) #changed form np.ones([1, t])
        K_plus = K_plus+t
    result = result[:, 0:K_plus]
    
    return list([result, K_plus])

def likelihood_opt_c1(double[:,:] X, double[:,:] Z, double sigma_A, double sigma_X, int K_plus, int num_objects, int object_dim):
    
    #cdef double[:,:] M
    #cdef double part1, part2, part3, part4, part5
        
    #Calculate M
    M = np.dot(Z.T, Z) + (sigma_X**2/sigma_A**2)*np.eye(K_plus)
    
    part1 = (-1)*num_objects*(0.5*object_dim)*log(2*pi)
    part2 = (-1)*(num_objects-K_plus)* object_dim *log(sigma_X) 
    part3 = (-1)*object_dim*K_plus*log(sigma_A) 
    part4 = (-1)*(0.5*object_dim)* log(np.linalg.det(M)) 
    part5 = (-1/(2*sigma_X**2)) * np.trace(np.dot(np.dot(X.T,(np.identity(num_objects) - np.dot(np.dot(Z,np.linalg.inv(M)),Z.T))),X))
    total = part1+part2+part3+part4+part5
    return(total)

def sampler_cython(image_data, num_objects, object_dim):
    
    #cdef int i, e, E
    #cdef double HN, alpha_N, p
    #cdef double[:, :] Z, Z_temp
    #cdef double alpha
    #cdef double[:] trunc, chain_K
    
    data = image_data

    #Set number of iterations
    E = 1000

    #Set truncation limit for max number of sampled latent features
    K_inf = 20

    #Set storage arrays for sampled parameters
    chain_Z = np.zeros([E, num_objects, K_inf])
    chain_K = np.zeros([E, 1])
    chain_sigma_X = np.zeros([E, 1])
    chain_sigma_A = np.zeros([E, 1])
    chain_alpha = np.zeros([E, 1])

    #Initialize parameter values
    sigma_X = 1
    sigma_A = 1
    alpha = 1
    num_object= np.shape(data)[0]
    object_dim = np.shape(data)[1]
    [Z, K_plus] = sampleIBP1(alpha, num_objects)

    #Compute Harmonic Number
    HN = 0
    for i in range(0, num_objects):
        HN = HN + 1/(i+1)

    for e in range(0, E):

        #Store sampled values
        chain_Z[e, :, 0:K_plus] = Z[:, 0:K_plus]
        chain_K[e] = K_plus
        chain_sigma_X[e] = sigma_X
        chain_sigma_A[e] = sigma_A
        chain_alpha[e] = alpha

        if (e%100==0):
            print(e)

        print("At iteration", e, ": K_plus is", K_plus, ", alpha is", alpha) 

        #Generate a new value for Z[i,k] and accept by Metropolis
        for i in range(0, num_objects):
            #First we remove singular features if any
            for k in range(0, K_plus):
                if (k>=K_plus):
                    break
                if(Z[i, k] > 0):
                    if (np.sum(Z[:, k]) - Z[i, k]) <= 0: 
                        Z[i, k] = 0
                        Z[:, k:(K_plus - 1)] = Z[:, (k+1):K_plus]
                        K_plus = K_plus - 1
                        Z = Z[:, 0:K_plus]
                        continue
                #Compute conditional distribution for current cell
                P=np.zeros(2)

                Z[i,k]=1
                P[0] = likelihood_opt_c1(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + log(np.sum(Z[:, k]) - Z[i,k]) - log(num_objects)

                Z[i,k]=0
                P[1] = likelihood_opt_c1(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + log(num_objects - np.sum(Z[:,k])) - log(num_objects)

                P = np.exp(P - np.max(P))

                #Sample from conditional(sampling previously sampled dishes)
                if np.random.uniform(0,1) < (P[0]/(np.sum(P))):
                    Z[i, k] = 1
                else:
                    Z[i, k] = 0

            #Sample new dishes by Metropolis
            trunc = np.zeros(5)
            alpha_N = alpha/num_objects

            for k_i in range(0,5):
                Z_temp = Z
                if k_i>0:
                    newcol = np.zeros((num_objects, k_i))
                    newcol[i,:] = 1 
                    Z_temp = np.column_stack((Z_temp, newcol))
                trunc[k_i] = k_i * log(alpha_N) - alpha_N - log(np.math.factorial(k_i)) + likelihood_opt_c1(data, Z_temp,sigma_A, sigma_X, K_plus+k_i, num_objects, object_dim)

            trunc = np.exp(trunc - np.max(trunc))
            trunc = trunc/np.sum(trunc)

            p = np.random.uniform(0,1)
            t = 0
            new_dishes = 0

            for k_i in range(0,5):
                t = t + trunc[k_i]
                if p < t:
                    new_dishes = k_i
                    break

            if(new_dishes > 0):
                newcol = np.zeros((num_objects, new_dishes))
                newcol[i,:] = 1
                Z = np.column_stack((Z, newcol))
            K_plus = K_plus + new_dishes

        #Sample sigma_X and sigma_A through Metropolis
        lik_curr = likelihood_opt_c1(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim)

        if np.random.uniform(0,1) < 0.5:
            sigma_X_new = sigma_X - np.random.uniform(0,1)/20
        else:
            sigma_X_new = sigma_X + np.random.uniform(0,1)/20

        lik_new_X = likelihood_opt_c1(data, Z, sigma_A, sigma_X_new, K_plus, num_objects, object_dim)

        acc_X = np.exp(min(0, lik_new_X - lik_curr))

        if np.random.uniform(0,1) < 0.5:
            sigma_A_new = sigma_A - np.random.uniform(0,1)/20
        else:
            sigma_A_new = sigma_A + np.random.uniform(0,1)/20

        lik_new_A = likelihood_opt_c1(data, Z, sigma_A_new, sigma_X, K_plus, num_objects, object_dim)

        acc_A = np.exp(min(0, lik_new_A - lik_curr))

        if np.random.uniform(0,1) < acc_X:
            sigma_X = sigma_X_new
        if np.random.uniform(0,1) < acc_A:
            sigma_A = sigma_A_new

        #Sample alpha via Gibbs
        alpha = gamma.rvs(a = 1 + K_plus, scale = 1/(1+HN))

    print("Complete")
    return(chain_K)

In [ ]:
sampler_cython(image_data, num_objects, object_dim)